# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest bs4 requests


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup
import os
import sys

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
def get_book_data(book_url: str) -> dict:
   """
   Парсит данные о книге со страницы сайта Books to Scrape.
    
   Args:
      book_url (str): URL страницы книги.
        
   Returns:
      dict: Словарь с информацией о книге, включающий название, цену, рейтинг, 
            наличие, описание и дополнительные характеристики из таблицы.
   """

   # НАЧАЛО ВАШЕГО РЕШЕНИЯ
   try:
      response = requests.get(book_url, timeout=10)
      response.raise_for_status() 
      soup = BeautifulSoup(response.content, 'html.parser')
   
      # Название
      title_element = soup.find('h1')
      title = title_element.text.strip() if title_element else 'Информация отсутствует'

      # Цена
      price_element = soup.find('p', class_='price_color')
      price = price_element.text.strip() if price_element else 'Информация отсутствует'
       
      # Рейтинг
      rating_element = soup.find('p', class_='star-rating')
      rating = 'Информация отсутствует'
      if rating_element:
         rating_classes = rating_element.get('class', [])
         rating_classes = [cls for cls in rating_classes if cls != 'star-rating']
         if rating_classes:
            rating = rating_classes[0].capitalize()

      # Наличие
      availability_element = soup.find('p', class_='instock availability')
      availability = 'Информация отсутствует'
      if availability_element:
         availability_text = availability_element.text.strip()
         availability = 'In stock' if 'In stock' in availability_text else 'Out of stock'

      # Описание
      description = 'Информация отсутствует'
      product_description = soup.find('div', id='product_description')
      if product_description:
         next_p = product_description.find_next('p')
         if next_p:
            description = next_p.text.strip()

      # Создание словаря с основной информацией о книге
      book_data = {
         'title': title,
         'price': price,
         'rating': rating,
         'availability': availability,
         'description': description,
      }

      # Обработка таблицы дополнительных характеристик и добавление их в словарь
      table = soup.find('table', class_='table table-striped')
      if table:
         rows = table.find_all('tr')
         for row in rows:
            header_element = row.find('th')
            value_element = row.find('td')
            if header_element and value_element:
               header = (
                  header_element.text.strip()
                  .lower()
                  .replace(' ', '_')
                  )
               value = value_element.text.strip()
               book_data[header] = value

      return book_data

   except requests.exceptions.HTTPError as e:
      print(f'HTTP ошибка для {book_url}: {e}')
      return {}
   except requests.exceptions.ConnectionError as e:
      print(f'Ошибка подключения к {book_url}: {e}')
      return {}
   except requests.exceptions.Timeout as e:
      print(f'Таймаут запроса к {book_url}: {e}')
      return {}
   except requests.exceptions.RequestException as e:
      print(f'Ошибка запроса к {book_url}: {e}')
      return {}
   except Exception as e:
      print(f'Неожиданная ошибка при обработке {book_url}: {e}')
      return {}
   # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 'Three',
 'availability': 'In stock (22 available)',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe 

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(is_save: bool = False) -> list:
    """
    Парсит книги со всех страниц сайта Books to Scrape.
    
    Args:
        is_save (bool): Если True, сохраняет данные в файл books_data.txt
        
    Returns:
        list: Список словарей с информацией о всех книгах
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
    page = 1
    list_books = []

    # Бесконечный цикл для обхода всех страниц
    while True:
        url = base_url.format(page)

        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            break
        except requests.exceptions.RequestException:
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        books = soup.find_all('article', class_='product_pod')
        if not books:
            break

        # Обрабатываем каждую книгу на текущей странице
        for book in books:
            link_tag = book.find('h3').find('a')
            if link_tag and link_tag.get('href'):
                link = link_tag['href']
                if link.startswith('../../../'):
                    book_url = link[9:]
                else:
                    book_url = link
                
                full_book_url = 'http://books.toscrape.com/catalogue/' + book_url
                book_data = get_book_data(full_book_url)
                if book_data:
                    list_books.append(book_data)

        # Проверяем наличие кнопки <next> для перехода на следующую страницу
        next_btn = soup.find('li', class_='next')
        if not next_btn:
            break
        page += 1

    # Если флаг сохранения установлен и есть данные для сохранения
    if is_save and list_books:
        try:
            file_path = 'D:/books_scraper/artifacts/books_data.txt'
            with open(file_path, 'w', encoding='utf-8') as f:
                for i, book in enumerate(list_books, 1):
                    book_text = f'Book №{i}\n'  
                    for key, value in book.items():
                        book_text += f'{key}: {value}\n'
                    f.write(book_text)
        except Exception as e:
            print(f'Ошибка при сохранении файла: {e}')

    return list_books 
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [7]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def schedule_scrape_books() -> None:
    """
    Автоматически парсит книги со всех страниц сайта Books to Scrape.
    Расписание:
    - Основной запуск: ежедневно в 19:00
    - Дополнительный тест (для проверки работоспособности программы): 
      каждые 30 минут
    """
    # Настройка ежедневного автоматического парсинга в 19:00
    schedule.every().day.at('19:00').do(scrape_books, is_save=True)

    # Проверки выполнения запланированных задач (каждые 60 секунд)
    while True:
        schedule.run_pending()
        time.sleep(60)

# if __name__ == "__main__":
#     schedule_scrape_books()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)
! pytest ../tests/test_scraper.py -v -s
#! pytest test/test_scraper.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-9.0.0, pluggy-1.6.0 -- d:\mipt\Python\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: d:\books_scraper
collecting ... collected 3 items

..\tests\test_scraper.py::test_get_book_structure PASSED
..\tests\test_scraper.py::test_get_book_key_value PASSED
..\tests\test_scraper.py::test_scrape_books__number_of_books PASSED

======================== 3 passed in 590.58s (0:09:50) ========================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```